In [ ]:
%matplotlib inline

import healpy as hp
import glob
from scipy.optimize import curve_fit
import pickle
from importlib import reload
import time
import scipy


# Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
import pysm
import qubic
from qubic import QubicSkySim as qss
from qubic import fibtools as ft
from qubic import camb_interface as qc
from qubic import SpectroImLib as si
from qubic import NamasterLib as nam
from qubic import mcmc

reload(qss)
reload(ft)

rc('figure', figsize=(16, 10))
rc('font', size=15)
mpl.rcParams['image.cmap'] = 'jet'

In [ ]:
### Initialize
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

dictfilename = global_dir + '/dicts/BmodesNoDustNoSystPaper0_2020.dict'


# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside']=256
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
print(center)

In [ ]:
#### This function reads the maps
def read_files_mc(dirmap,file_rnd_string, verbose=False):
    m = np.array(FitsArray(glob.glob(dirmap+'*_maps_recon_*'+file_rnd_string+'.fits')[0]))
    c = np.array(FitsArray(glob.glob(dirmap+'*_maps_convolved_*'+file_rnd_string+'.fits')[0]))
    cov = np.array(FitsArray(glob.glob(dirmap+'*_maps_coverage_*'+file_rnd_string+'.fits')[0]))
    with open(glob.glob(dirmap+'*_dictionary_'+file_rnd_string+'.pickle')[0], 'rb') as handle: d = pickle.load(handle)
    with open(glob.glob(dirmap+'*_input_cell_'+file_rnd_string+'.pickle')[0], 'rb') as handle: icl = pickle.load(handle)
    filetype = glob.glob(dirmap+'*_maps_recon_*'+file_rnd_string+'.fits')[0].split('/')[-1]
    if verbose: print('read {}'.format(filetype))
    return m, c, cov, d, icl, filetype

### Get reconstructed maps from a simulation made on NERSC with 200000 pointings and tol=1e-5
### First Instrument
### detector_nep=4.7e-17
### effecgtive_duration = 3 years
nptg = 200000
dirmaps='/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB_NERSC/Maps/'
file_rnd_string='H990hwzaGn'
tol=1e-5

m, c, cov, d, icl, ftype = read_files_mc(dirmaps, file_rnd_string, verbose=True)

nbins = 100   
reload(qss)
xx, myY, effective_variance_invcov = qss.get_noise_invcov_profile(m[0,:,:]-c[0,:,:], 
                                                                  cov, 0.1, nbins=nbins, doplot=True,
                                                                  fit=True, norm=True, allstokes=True,
                                                                 label='QUBIC MC Average IQU Nptg={}  Tol={}'.format(nptg,tol))
plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:]),'--')
legend()

In [ ]:
### Now do a realization of QUBIC sky
reload(qss)
reload(qc)
## Make a sky using PYSM: It will have the expected QUBIC beam, the coverage and noise according to this coverage
seed = None
sky_config = {'cmb': seed}
Qubic_sky = qss.Qubic_sky(sky_config, d)

signoise = 75
qubicnoise = Qubic_sky.create_noise_maps(signoise, cov, effective_variance_invcov=effective_variance_invcov)

xx, yyfs, bla = qss.get_noise_invcov_profile(qubicnoise, cov, label='FastSim', fit=False, allstokes=True,doplot=True)
xx, yysim, bla = qss.get_noise_invcov_profile(m[0,:,:]-c[0,:,:], cov, 
                                   label='QUBIC MC Average IQU Nptg={}  Tol={}'.format(nptg,tol), fit=False, doplot=True)
plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:])*yysim[0],'--',label='Qubic Law')
xlim(0,12)
ylim(0,2.5)
legend()

In [ ]:
reload(qss)
qubicnoise = Qubic_sky.create_noise_maps(signoise, cov, effective_variance_invcov=effective_variance_invcov)

dirmapsold = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Maps/'
rndstr = 'nXR0M957gz'
mold = np.array(FitsArray(dirmapsold + 'CMB-Only-12sub_3000_1e-5_maps_recon_seed_None_'+rndstr+'.fits'))
covold = np.array(FitsArray(dirmapsold + 'CMB-Only-12sub_3000_1e-5_maps_coverage_'+rndstr+'.fits'))
cold = np.array(FitsArray(dirmapsold + 'CMB-Only-12sub_3000_1e-5_maps_convolved_seed_None_'+rndstr+'.fits'))

rc('figure', figsize=(16, 10))
dofit=True
nbins = 80
xx, yy, fit_fs = qss.get_noise_invcov_profile(qubicnoise, cov, nbins=nbins,
                                              label='FastSim from NERSC 200000',fit=dofit, doplot=True)
xx, yy, fit_n200k = qss.get_noise_invcov_profile(m[0,:,:]-c[0,:,:], cov, nbins=nbins,
                                              label='NERSC 200000',fit=dofit, doplot=True)
xx, yy, fit_n3k = qss.get_noise_invcov_profile(mold[0,:,:]-cold[0,:,:], covold, nbins=nbins,
                                              label='MacQUBIC 3000',fit=dofit, doplot=True)
legend()

print(qubicnoise.shape)
rc('figure', figsize=(16, 10))
res = 15
figure()
st = ['I', 'Q', 'U']
rng = [5, 5, 5]
for s in range(3):
    hp.gnomview(qubicnoise[:,s], rot=center, reso=res, sub=(3,3,3*s+1), title='FastSim '+st[s],
               min=-rng[s], max=rng[s])
    hp.gnomview(m[0,:,s]-c[0,:,s], rot=center, reso=res, sub=(3,3,3*s+2), title='NERSC 200000 '+st[s],
               min=-rng[s], max=rng[s])
    hp.gnomview(mold[0,:,s]-cold[0,:,s], rot=center, reso=res, sub=(3,3,3*s+3), title='NERSC 3000 '+st[s],
               min=-rng[s], max=rng[s])
tight_layout()


rc('figure', figsize=(16, 7))
figure()
st = ['I', 'Q', 'U']
diff = m[0,:,:]-c[0,:,:]
diffold = mold[0,:,:]-cold[0,:,:]
bins=50
lims = 25
for s in range(3):
    subplot(1,3,s+1)
    okpix = (qubicnoise[:,s] != 0) & (diff[:,s] != 0) & (diffold[:,s] != 0)
    hist(diffold[okpix,s], range=[-lims,lims], bins=bins, histtype='step', 
         label = 'NERSC 3000 '+st[s]+': '+ft.statstr(diffold[okpix,s]))
    hist(qubicnoise[okpix,s], range=[-lims,lims], bins=bins, histtype='step',
         label = 'FastSim '+st[s]+': '+ft.statstr(qubicnoise[okpix,s]))
    hist(diff[okpix,s], range=[-lims,lims], bins=bins, histtype='step', 
         label = 'NERSC 200000 '+st[s]+': '+ft.statstr(diff[okpix,s]))
    yscale('log')
    legend(fontsize=10, loc='lower right')
tight_layout()

## Exploration of IQU Correlations

In [ ]:
# ### Let's look at covariances between I,Q,U...
reload(qss)
reload(ft)

# 1) Correct maps for RMS vs 1/cov
new_qubicnoise = qss.correct_maps_rms(qubicnoise, cov, effective_variance_invcov)
new_diff = qss.correct_maps_rms(diff, cov, effective_variance_invcov)
new_diffold = qss.correct_maps_rms(diffold, covold, fit_n3k)


#### Check that it's flat coverage...
subplot(1,3,1)
bla = qss.get_angular_profile(new_qubicnoise, label='Fastsim corrected',
                              allstokes=True,nbins=100,thmax=22, fontsize=10, doplot=True)
subplot(1,3,2)
bla = qss.get_angular_profile(new_diff, label='Nersc 200k corrected',
                              allstokes=True,nbins=100,thmax=22, fontsize=10, doplot=True)
subplot(1,3,3)
bla = qss.get_angular_profile(new_diffold, label='Nersc 3k corrected',
                              allstokes=True,nbins=100,thmax=22, fontsize=10, doplot=True)

# 2) Calculate covariance matrix averaging over pixels
okpix = new_qubicnoise[:,0] > 0
cc_fs = np.corrcoef(new_qubicnoise[okpix,:].T)

okpix = new_diff[:,0] > 0
cc_diff = np.corrcoef(new_diff[okpix,:].T)

okpix = new_diffold[:,0] > 0
cc_diffold = np.corrcoef(new_diffold[okpix,:].T)

figure()
rc('figure', figsize=(16, 4))
vv = 0.05
subplot(1,3,1)
imshow(np.abs(cc_fs-np.diag([1,1,1])), vmin=0, vmax=vv)
plt.xticks(range(3), ['I','Q','U'])
plt.yticks(range(3), ['I','Q','U'])
title('FastSim')
colorbar()
subplot(1,3,2)
imshow(np.abs(cc_diff-np.diag([1,1,1])), vmin=0, vmax=vv)
plt.xticks(range(3), ['I','Q','U'])
plt.yticks(range(3), ['I','Q','U'])
title('Nersc 200k')
colorbar()
subplot(1,3,3)
imshow(np.abs(cc_diffold-np.diag([1,1,1])), vmin=0, vmax=vv)
plt.xticks(range(3), ['I','Q','U'])
plt.yticks(range(3), ['I','Q','U'])
title('Nersc 3k')
colorbar()
tight_layout()

print('Fast Simulation')
print(cc_fs)
print('Nersc 200k')
print(cc_diff)
print('3K')
print(cc_diffold)


It is better to have MC for FastNoise so that we have an. expected dispersion to compare with:

In [ ]:
nbmc = 100

all_cc_fs = np.zeros((nbmc, 3,3))

for i in range(nbmc):
    qubicnoise = Qubic_sky.create_noise_maps(signoise, cov, effective_variance_invcov=effective_variance_invcov)
    new_qubicnoise = qss.correct_maps_rms(qubicnoise, cov, effective_variance_invcov)
    okpix = new_qubicnoise[:,0] > 0
    all_cc_fs[i,:,:] = np.corrcoef(new_qubicnoise[okpix,:].T)


In [ ]:
m_cc_fs = np.mean(all_cc_fs, axis=0)
s_cc_fs = np.std(all_cc_fs, axis=0)
print('MC Average')
print(m_cc_fs)
print('MC STD')
print(s_cc_fs)
print('Nersc 200k')
print(cc_diff)

figure()
subplot(1,2,1)
imshow(np.log10(np.abs(cc_diff)), vmin=-3, vmax=0)
plt.xticks(range(3), ['I','Q','U'])
plt.yticks(range(3), ['I','Q','U'])
title('Nersc 200k')
colorbar()

subplot(1,2,2)
imshow(np.log10(np.abs(s_cc_fs)), vmin=-3, vmax=0)
plt.xticks(range(3), ['I','Q','U'])
plt.yticks(range(3), ['I','Q','U'])
title('RMS Monte-Carlo')
colorbar()

figure()
subplot(1,2,1)
imshow(np.log10(np.abs(cc_diffold)), vmin=-3, vmax=0)
plt.xticks(range(3), ['I','Q','U'])
plt.yticks(range(3), ['I','Q','U'])
title('3K')
colorbar()

subplot(1,2,2)
imshow(np.log10(np.abs(s_cc_fs)), vmin=-3, vmax=0)
plt.xticks(range(3), ['I','Q','U'])
plt.yticks(range(3), ['I','Q','U'])
title('RMS Monte-Carlo')
colorbar()


There is a marginally significant correlation between Q and U that is currently neglected. Note that this is done with a single realization and would therefore require to have a look at a number of realizations.

## Exploration of spatial Correlations within a map

In [ ]:
### Let's try to look at covariance between nearby pixels
reload(qss)
reload(ft)

new_qubicnoise = qss.correct_maps_rms(qubicnoise, cov, fit_n200k)
new_diff = qss.correct_maps_rms(diff, cov, fit_n200k)
new_diff3k = qss.correct_maps_rms(diffold, covold, fit_n3k)


subplot(1,3,1)
bla = qss.get_angular_profile(new_qubicnoise, label='Fastsim corrected',
                              allstokes=True,nbins=50,thmax=22, fontsize=10, doplot=True)
ylim(0,1.2)
xlim(0,22)
title('FastSim')

subplot(1,3,2)
bla = qss.get_angular_profile(new_diff, label='Nersc 200k corrected',
                              allstokes=True,nbins=50,thmax=22, fontsize=10, doplot=True)
ylim(0,1.2)
xlim(0,22)
title('200k')

subplot(1,3,3)
bla = qss.get_angular_profile(new_diff3k, label='Nersc 3k corrected',
                              allstokes=True,nbins=50,thmax=22, fontsize=10, doplot=True)
ylim(0,1.2)
xlim(0,22)
title('3k')


In [ ]:
ns = d['nside']
okpix = cov > (np.max(cov)*0.1)
ip = np.arange(12*ns**2).astype(int)
ipok = ip[okpix]

th, cth_qn = qss.ctheta_parts(new_qubicnoise[:,0], ipok, 0, 20, 20, nsplit=5, degrade_init=128, verbose=False)
th, cth_200k = qss.ctheta_parts(new_diff[:,0], ipok, 0, 20, 20, nsplit=5, degrade_init=128, verbose=False)
th, cth_3k = qss.ctheta_parts(new_diff3k[:,0], ipok, 0, 20, 20, nsplit=5, degrade_init=128, verbose=False)


In [ ]:
plot(th,(cth_qn),'o', label='FastSim')
plot(th,(cth_3k),'o', label ='3k')
plot(th,(cth_200k),'o', label='200k')
plot(th,th*0,'k:')
legend()
xlabel(r'$\theta$ [deg]')
ylabel(r'$C(\theta)$')


In [ ]:
from scipy.optimize import curve_fit

fct = lambda x, a, b, c: a * np.sin(x/b) * exp(-x/c)
results_qn = curve_fit(fct, th[1:], (cth_qn[1:]/cth_qn[0]),maxfev=100000, ftol=1e-7, p0=[0,1,1])
results_3k = curve_fit(fct, th[1:], (cth_3k[1:]/cth_3k[0]),maxfev=100000, ftol=1e-7, p0=[0,1,1])
results_200k = curve_fit(fct, th[1:], (cth_200k[1:]/cth_200k[0]),maxfev=100000, ftol=1e-7, p0=[0,1,1])

thth = np.linspace(0,20,100)
p=plot(th,(cth_qn/cth_qn[0]),'o', label='FastSim')
plot(thth, fct(thth, *results_qn[0]), color=p[0].get_color(),
     label=r'$a\times\sin(\theta/b)\times\exp(\theta/c)$'+'\n a={0:4.2f} b={1:4.2f} c={2:4.2f}'.format(*results_qn[0]))
p=plot(th,(cth_3k/cth_3k[0]),'o', label ='3k')
plot(thth, fct(thth, *results_3k[0]), color=p[0].get_color(),
     label=r'$a\times\sin(\theta/b)\times\exp(\theta/c)$'+'\n a={0:4.2f} b={1:4.2f} c={2:4.2f}'.format(*results_3k[0]))

p=plot(th,(cth_200k/cth_200k[0]),'o', label='200k')
plot(thth, fct(thth, *results_200k[0]), color=p[0].get_color(),
     label=r'$a\times\sin(\theta/b)\times\exp(\theta/c)$'+'\n a={0:4.2f} b={1:4.2f} c={2:4.2f}'.format(*results_200k[0]))

legend(loc='upper right',fontsize=10)
xlabel(r'$\theta$ [deg]')
ylabel(r'$C(\theta)$ [Normalized]')
ylim(-0.1,1)


Now go to Cl space

In [ ]:
rc('figure', figsize=(15, 6))
reload(qc)
reload(qss)

mylmax = 1024
theta = np.linspace(0,180,1000)
ctheta = fct(theta, *results_200k[0])
ctheta[0] = 1
#ctheta[1:] = 0

#### Disply input C(theta)
subplot(1,2,1)
plot(theta,ctheta,label='Input C(theta)')
xlabel(r'$\theta$ [deg]')
ylabel(r'$C(\theta)$')
xlim(0,25)

#### COnvert to Cl and display
lll, clth = qc.ctheta_2_cell(theta, ctheta, lmax=mylmax)

#### Go back to c(theta)
thback, ctheta_back = qc.cell_2_ctheta(clth, theta_deg=theta)
plot(theta, ctheta_back, '--',label='C(theta) back from Cell')
plot(theta, theta*0,'k:')
legend()


subplot(1,2,2)
plot(lll, clth, label='Output Cell / cl[0]')
plot(lll, lll*0+1,'k:')
xlabel(r'$\ell$')
ylabel(r'$C_\ell$')
xlim(0,np.max(lll))
legend()


In [ ]:
### Analyse results (similarly as in Analyze_ClCross_images)
def read_one(dircls,lm,dl,cc, verbose=False, name='res'):
    filename = dircls+name+'_{0:}_{1:}_{2:}.pickle'.format(lm, dl, cc)
    ff = glob.glob(filename)
    if len(ff)!=1:
        print('Problem with '+filename)
        return -1,-1,-1
    else:
        ### Read file
        with open(ff[0], 'rb') as handle:
            leff, mcl,scl = pickle.load(handle)
        return leff,mcl,scl

def read_sims(dircls,lm,dl,cc, verbose=False, type='best'):
    if type=='flat':
        leff, mcl, scl = read_one(dircls, lm, dl, cc, verbose=verbose, name='res')
    elif type=='cov':
        leff, mcl, scl = read_one(dircls, lm, dl, cc, verbose=verbose, name='rescov')
    elif type=='best':
        leff, mcl_flat, scl_flat = read_one(dircls, lm, dl, cc, verbose=verbose, name='res')
        leff, mcl_cov, scl_cov = read_one(dircls, lm, dl, cc, verbose=verbose, name='rescov')
    mcl = np.zeros_like(mcl_flat)
    scl = np.zeros_like(scl_flat)
    for i in range(len(leff)):
        for s in range(4):
            if scl_flat[i,s] <= scl_cov[i,s]:
                mcl[i,s] = mcl_flat[i,s]
                scl[i,s] = scl_flat[i,s]
            else:
                mcl[i,s] = mcl_cov[i,s]
                scl[i,s] = scl_cov[i,s]
    return leff, mcl, scl


def ana_likelihood(rv, leff, fakedata, errors, model, prior, 
                   mylikelihood=mcmc.LogLikelihood, covariance_model_funct=None, otherp=None):
    ll = mylikelihood(xvals=leff, yvals=fakedata, errors=errors, 
                            model = model, flatprior=prior, covariance_model_funct=covariance_model_funct)  
    like = np.zeros_like(rv)
    for i in range(len(rv)):
        like[i] = np.exp(ll([rv[i]]))
    cumint = scipy.integrate.cumtrapz(like, x=rv)
    cumint = cumint / np.max(cumint)
    onesigma = np.interp(0.68, cumint, rv[1:])
    if otherp:
        other = np.interp(otherp, cumint, rv[1:])
        return like, cumint, onesigma, other
    else:
        return like, cumint, onesigma


def explore_like(leff, mcl_noise, scl_noise, lmin, dl, cc, rv, otherp=None,
                 cov=None, plotlike=False, plotcls=False, 
                 verbose=False, sample_variance=True, mytitle='', color=None, mylabel='',my_ylim=None):
    
#     print(lmin, dl, cc)
#     print(leff)
#     print(scl_noise[:,2])
    ### Create Namaster Object
    # Unfortunately we need to recalculate fsky for calculating sample variance
    nside=256
    if cov is None:
        Namaster = nam.Namaster(None, lmin=lmin, lmax=lmax, delta_ell=dl)
        Namaster.fsky = 0.018
    else:
        okpix = cov > (np.max(cov) * float(cc))
        maskpix = np.zeros(12*nside**2)
        maskpix[okpix] = 1
        Namaster = nam.Namaster(maskpix, lmin=lmin, lmax=lmax, delta_ell=dl)
    
    print('Fsky: {}'.format(Namaster.fsky))
    lbinned, b = Namaster.get_binning(nside)

    ### Bibnning CambLib
    binned_camblib = qc.bin_camblib(Namaster, '../../scripts/QubicGeneralPaper2020/camblib.pickle', 
                                    nside, verbose=False)

    ### Redefine the function for getting binned Cls
    def myclth(ell,r):
        clth = qc.get_Dl_fromlib(ell, r, lib=binned_camblib, unlensed=False)[0]
        return clth
    allfakedata = myclth(leff, 0.)
    
    ### And we need a fast one for BB only as well
    def myBBth(ell, r):
        clBB = qc.get_Dl_fromlib(ell, r, lib=binned_camblib, unlensed=False, specindex=2)[0]
        return clBB

    ### Fake data
    fakedata = myBBth(leff, 0.)        
    
    if sample_variance:
        covariance_model_funct = Namaster.knox_covariance
    else:
        covariance_model_funct = None
    if otherp is None:
        like, cumint, allrlim = ana_likelihood(rv, leff, fakedata, 
                                            scl_noise[:,2], 
                                            myBBth, [[0,1]],
                                           covariance_model_funct=covariance_model_funct)
    else:
        like, cumint, allrlim, other = ana_likelihood(rv, leff, fakedata, 
                                            scl_noise[:,2], 
                                            myBBth, [[0,1]],
                                           covariance_model_funct=covariance_model_funct, otherp=otherp)
    
    if plotcls:
        if plotlike:
            subplot(1,2,1)
        #plot(inputl, inputcl[:,2], 'k', label='r=0')
        plot(leff, scl_noise[:,2], label=mylabel+' Errors', color=color)
        xlim(0,lmax)
        if my_ylim is None:
            ylim(1e-4,1e0)
        else:
            ylim(my_ylim[0], my_ylim[1])
        yscale('log')
        xlabel('$\\ell$')
        ylabel('$D_\\ell$')
        legend(loc='upper left')
    if plotlike:
        if plotcls:
            subplot(1,2,2)
        p=plot(rv, like/np.max(like), 
               label=mylabel+' $\sigma(r)={0:6.4f}$'.format(allrlim), color=color)
        plot(allrlim+np.zeros(2), [0,1.2], ':', color=p[0].get_color())
        xlabel('r')
        ylabel('posterior')
        legend(fontsize=8, loc='upper right')
        xlim(0,0.1)
        ylim(0,1.2)
        title(mytitle)
    
    if otherp is None:
        return like, cumint, allrlim
    else:
        return like, cumint, allrlim, other



In [ ]:
#### To read the end-to-end sims made on MacQubic with 3000 ptg
def read_end2end(dircls, k,lm,dl,cc, verbose=True):
    filename = dircls+'cls_output_k_{0:}_ellmin_{1:}_dl_{2:}_covcut_{3:}.pickle'.format(k,lm, dl, cc)
    ff = glob.glob(filename)
    #print(ff,len(ff))
    if len(ff)<1:
        if verbose: print('Problem with '+filename)
        return -1
    else:
        ### Read file
        with open(ff[0], 'rb') as handle:
            leff, allclsFlat, allclsCov, allpairs, inputcl, inputl = pickle.load(handle)
            mcls_flat = np.mean(allclsFlat,axis=0)
            scls_flat = np.std(allclsFlat,axis=0)
            mcls_cov = np.mean(allclsCov,axis=0)
            scls_cov = np.std(allclsCov,axis=0)
            mcls_best = np.zeros((len(leff),4))
            scls_best = np.zeros((len(leff),4))
            for i in range(len(leff)):
                for s in range(4):
                    if scls_flat[i,s] <= scls_cov[i,s]:
                        mcls_best[i,s] = mcls_flat[i,s]
                        scls_best[i,s] = scls_flat[i,s]
                    else:
                        mcls_best[i,s] = mcls_cov[i,s]
                        scls_best[i,s] = scls_cov[i,s]
        return leff, mcls_flat, scls_flat, mcls_cov, scls_cov, mcls_best, scls_best, inputcl, inputl

camblib = qc.read_camblib('../../scripts/QubicGeneralPaper2020/camblib.pickle')    
rv = np.linspace(0,1.,1000)
lmax = 2*256+1
inputcl = Qubic_sky.input_cmb_spectra
lth = np.arange(len(inputcl[:,0]))
plnames = ['TT', 'EE', 'BB']


dir_end2end = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Cls/'
k = 'CMB-Only-12sub_3000_1e-5_s_42'
lm3k = 15
dl3k = 60
cc3k = 0.2
leff_3k, mcl_f3k, scl_f3k, mcl_c3k, scl_c3k, mcl_b3k, scl_b3k, icell, iell = read_end2end(dir_end2end,
                                                                                        k, lm3k,dl3k,cc3k)
cov3k = np.array(FitsArray('/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Maps/CMB-Only-12sub_3000_1e-5_maps_coverage_zR2hSgphTA.fits'))
bla = explore_like(leff_3k, scl_b3k, scl_b3k, lm3k, dl3k, cc3k, rv,
                 cov=cov3k, plotlike=True, plotcls=True, 
                 verbose=True, sample_variance=True)



So now we can perform many simulations of the same...

In [ ]:
#### Now in a function to loop over ell binning, lmin, coverage
def run_mc(nbmc, Namaster, d, signoise, cov, effective_variance_invcov, verbose=False, clnoise=None):
    ell_bins, b = Namaster.get_binning(d['nside'])
    mask_apo = Namaster.mask_apo
    okpix = cov > (np.max(cov)*0.1)

    myd = d.copy()
    myd['nf_sub']=1
    seed = np.random.randint(1,100000)
    sky_config = {'cmb': seed}
    Qubic_sky = qss.Qubic_sky(sky_config, myd)

    w=None
    cl_noise_qubic = np.zeros((nbmc, len(ell_bins), 4))
    print('        Starting MC')
    for imc in range(nbmc):
        t0 = time.time()
        qubicnoiseA = Qubic_sky.create_noise_maps(signoise, cov, 
                                                  effective_variance_invcov=effective_variance_invcov,
                                                 clnoise=clnoise, Nyears=3)
#         qubicnoiseA[~okpix] = 0
#         qubicnoiseA[okpix] -= np.mean(qubicnoiseA[okpix])
        qubicnoiseB = Qubic_sky.create_noise_maps(signoise, cov, 
                                                  effective_variance_invcov=effective_variance_invcov,
                                                 clnoise=clnoise, Nyears=3)
#         qubicnoiseB[~okpix] = 0
#         qubicnoiseB[okpix] -= np.mean(qubicnoiseB[okpix])
        
#         figure()
#         plot(effective_variance_invcov[0,:], np.sqrt(effective_variance_invcov[1,:]), label='Norm Profile')
#         bla = qss.get_noise_invcov_profile(qubicnoiseA, cov, label='A not norm',fit=False)
#         bla = qss.get_noise_invcov_profile(qubicnoiseB, cov, label='B not norm',fit=False)
        
        ### Compute Spectra:
        # Noise Only
        if verbose: print('   - QUBIC Noise maps')
        leff, cl_noise_qubic[imc, :,:], w = Namaster.get_spectra(qubicnoiseA.T, 
                                                                 map2 = qubicnoiseB.T,
                                                                 purify_e=False, purify_b=True, w=w, verbose=False,
                                                                 beam_correction=True)
        t1 = time.time()
        print('             Monte-Carlo: Iteration {0:} over {1:} done in {2:5.2f} sec'.format(imc, nbmc,t1-t0))
        
    
    # average MC results
    mcl_noise_qubic = np.mean(cl_noise_qubic, axis=0)
    scl_noise_qubic = np.std(cl_noise_qubic, axis=0)
    return leff, mcl_noise_qubic, scl_noise_qubic
    



In [ ]:
reload(qss)
dofit=True
nbins = 20
xx, yy, fit_n200k = qss.get_noise_invcov_profile(m[0,:,:]-c[0,:,:], cov, nbins=nbins,
                                              label='NERSC 200000',fit=dofit, covcut=0.1, doplot=True)
xx, yy, fit_n3k = qss.get_noise_invcov_profile(mold[0,:,:]-cold[0,:,:], covold, nbins=nbins,
                                              label='MacQUBIC 3000',fit=dofit, covcut=0.1, doplot=True)

plot(fit_n3k[0,:], np.sqrt(fit_n3k[1,:]), label='3k')
plot(fit_n200k[0,:], np.sqrt(fit_n200k[1,:]), label='200k')
xlim(0,11)
ylim(0,3)
legend(loc='upper left', fontsize=8)

In [ ]:
alpha = 4.5 #### See Notebook called "2pt-Correlation Function" for an empirical explanation of the factor alpha
newclth = (clth-1)*alpha+1
plot(clth, label='Initial')
plot(newclth, label='Renormalized')
plot(clth*0+1,'k:')
legend()

In [ ]:
from qubic import NamasterLib as nam
reload(qss)
reload(qc)
nbmc = 10
signoise = 75.   # initial was 75.

# Create a Namaster object
lmin = 15
lmax = 2 * d['nside'] - 1
delta_ell = 60
covcut = 0.2

okpix = cov > np.max(cov) * covcut

### We use Flat weighting
maskpix = np.zeros(12*d['nside']**2)
maskpix[okpix] = 1
Namaster = nam.Namaster(maskpix, lmin=lmin, lmax=lmax, delta_ell=delta_ell)

leff, mcl_noise_qubic, scl_noise_qubic = run_mc(nbmc, Namaster, d, signoise, cov, fit_n200k, clnoise=newclth)

In [ ]:
lm3k = lmin
dl3k = delta_ell
cc3k = covcut
leff_3k, mcl_f3k, scl_f3k, mcl_c3k, scl_c3k, mcl_b3k, scl_b3k, icell, iell = read_end2end(dir_end2end,
                                                                            'CMB-Only-12sub_3000_1e-5_s_42',
                                                                            lm3k,dl3k,cc3k, verbose=True)
subplot(1,2,1)
plot(leff, scl_noise_qubic[:,2]/leff/(leff+1),label='FastNoise')
plot(leff, scl_f3k[:,2]/leff/(leff+1), label='3k')
yscale('log')
legend()

subplot(1,2,2)
plot(leff, scl_noise_qubic[:,2] / scl_b3k[:,2])
plot(leff, leff*0+1,'k:')


In [ ]:
### Analyse MC
rc('figure', figsize=(16, 6))
input_cell = Qubic_sky.input_cmb_spectra
lth = np.arange(len(input_cell[:,0]))
plnames = ['TT', 'EE', 'BB']

print(leff)
for i in range(3):
    subplot(1,3,i+1)
    xlim(0,2*d['nside'])
    ylim(0,np.max(input_cell[:,i])*1.5)
    plot(lth, input_cell[:,i], 'k')
    #errorbar(leff, mcl_noise_qubic[:,i], yerr=scl_noise_qubic[:,i], fmt='bo', label='Noise: QUBIC Coverage')
    plot(leff, scl_noise_qubic[:,i],'b', label='FastSim')
    plot(leff_3k, scl_f3k[:,i], 'r', label='e2e 3k')
    title(plnames[i])
    yscale('log')
    ylim(1e-4,1e4)
    legend(fontsize=9)
    
figure()
like, cumint, rlim = explore_like(leff, scl_noise_qubic, scl_noise_qubic, lmin, delta_ell, covcut, rv,
                 cov=cov, plotlike=True, plotcls=True, 
                 verbose=True, sample_variance=True, mytitle='FastSim', color='b')

figure()
dir_end2end = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Cls/'
k = 'CMB-Only-12sub_3000_1e-5_s_42'
lm3k = lmin*1
dl3k = delta_ell*1
cc3k = covcut*1
leff_3k, mcl_f3k, scl_f3k, mcl_c3k, scl_c3k, mcl_b3k, scl_b3k, icell, iell = read_end2end(dir_end2end,
                                                                                        k, lm3k,dl3k,cc3k)
cov3k = np.array(FitsArray('/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Maps/CMB-Only-12sub_3000_1e-5_maps_coverage_zR2hSgphTA.fits'))
bla = explore_like(leff_3k, scl_f3k*0, scl_f3k, lm3k, dl3k, cc3k, rv,
                 cov=cov3k, plotlike=True, plotcls=True, 
                 verbose=True, sample_variance=True, mytitle='e2e 3K', color='r')

title(r'$\ell_m = {} \;-\; \Delta_\ell = {} \;-\; Cov_c = {}$'.format(lmin, delta_ell, covcut))

subplot(1,2,1)
plot(leff, scl_noise_qubic[:,2],'b', label='FastSim alpha=0.45 on Mac')
plot(leff_3k, scl_f3k[:,2],'r', label='e2e 3K Flat')
legend()
yscale('log')
ylim(5e-4, 0.2)

title(r'$\ell_m = {} \;-\; \Delta_\ell = {} \;-\; Cov_c = {}$'.format(lmin, delta_ell, covcut))

subplot(1,2,2)
plot(rv, like, 'b', label='FastSim Flat $\sigma(r)$={0:5.4f}'.format(rlim))
legend()

In [ ]:
#### Need to ssave. these files for simulations in Lyon
# pickle.dump(clth, open('cl_corr_noise_nersc200k.pk', "wb"))
# FitsArray(cov).save('Coverage_Nersc200k.fits')
# pickle.dump(clth, open('cl_corr_noise_nersc200k.pk', "wb"))

In [ ]:
#### Reading sims from Lyon
### Analyse results (similarly as in Analyze_ClCross_images)
def read_one_lyon(dircls,prefix, lm,dl,cc, verbose=False, name='_Flat'):
    filename = dircls+prefix+'_lmin_{0:}_dl_{1:}_cc_{2:}'.format(lm, dl, cc)+name+'.pk'
    ff = glob.glob(filename)
    if len(ff)!=1:
        print('Problem with '+filename)
        return -1,-1,-1
    else:
        ### Read file
        with open(ff[0], 'rb') as handle:
            leff, mcl,scl, _ = pickle.load(handle)
        return leff,mcl,scl

def read_sims_lyon(dircls,prefix,lm,dl,cc, verbose=False, type='best'):
    print(lm,dl,cc)
    if type=='flat':
        leff, mcl, scl = read_one_lyon(dircls, prefix, lm, dl, cc, verbose=verbose, name='_Flat')
        return leff, mcl, scl
    elif type=='cov':
        leff, mcl, scl = read_one_lyon(dircls, prefix, lm, dl, cc, verbose=verbose, name='_Cov')
        return leff, mcl, scl
    elif type=='best':
        leff, mcl_flat, scl_flat = read_one_lyon(dircls, prefix, lm, dl, cc, verbose=verbose, name='_Flat')
        leff, mcl_cov, scl_cov = read_one_lyon(dircls, prefix, lm, dl, cc, verbose=verbose, name='_Cov')
    mcl = np.zeros_like(mcl_flat)
    scl = np.zeros_like(scl_flat)
    for i in range(len(leff)):
        for s in range(4):
            if scl_flat[i,s] <= scl_cov[i,s]:
                mcl[i,s] = mcl_flat[i,s]
                scl[i,s] = scl_flat[i,s]
            else:
                mcl[i,s] = mcl_cov[i,s]
                scl[i,s] = scl_cov[i,s]
    return leff, mcl, scl

rc('figure', figsize=(15, 6))
rc('font', size=12)
reload(qc)
reload(nam)
reload(mcmc)
import scipy

### Read the camb lib from the file - We will not use it directly but it will 
### be binned according to the. binning scheme for each case
camblib = qc.read_camblib('../../scripts/QubicGeneralPaper2020/camblib.pickle')    
rv = np.linspace(0,1.,1000)
    
lm = 15
dl = 60
cc = 0.2

all_dirsims = []
all_prefix = []
all_titles = []

#### Simus sans corrélation spatiale
dirsims = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/FastSimsNoiseLyon/FastSimNoSpaceCorr/'
prefix = 'MC_MCFastNoise_n_300_sig_75.0'
all_dirsims.append(dirsims)
all_prefix.append(prefix)
all_titles.append('No Space Correlation')

#### Simus avec alpha = 1
# dirsims = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/FastSimsNoiseLyon/FastSimAlpha_1/'
# prefix = 'MC_MCFastNoise_n_1000_sig_75.0'
# all_dirsims.append(dirsims)
# all_prefix.append(prefix)
# all_titles.append('alpha=1')

#### Simus avec Alpha = 4.5
dirsims = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/FastSimsNoiseLyon/FastSimAlpha_4.5/'
prefix = 'MC_MCFastNoise_n_1000_sig_75.0'
all_dirsims.append(dirsims)
all_prefix.append(prefix)
all_titles.append('alpha=4.5')


for i in range(len(all_dirsims)):
    dirsims = all_dirsims[i]
    prefix = all_prefix[i]
    leff,mcl,scl = read_sims_lyon(dirsims, prefix, 
                                    lm, dl, cc, type='flat')
    like, cumint, rlim = explore_like(leff, scl*0, scl, lm, dl, cc, rv,
                                     cov=cov, plotlike=True, plotcls=True, 
                                     verbose=True, sample_variance=True, mylabel=all_titles[i])

#### Get End-To-End sims 3K ##############################
k = 'CMB-Only-12sub_3000_1e-5_s_42'
lm3k = lm*1
dl3k = dl*1
cc3k = cc*1
leff_3k, mcl_f3k, scl_f3k, mcl_c3k, scl_c3k, mcl_b3k, scl_b3k, icell, iell = read_end2end(dir_end2end,
                                                                                        k, lm3k,dl3k,cc3k)
cov3k = np.array(FitsArray('/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Maps/CMB-Only-12sub_3000_1e-5_maps_coverage_zR2hSgphTA.fits'))

like_3k, cumint_3k, rlim_3k = explore_like(leff_3k, scl_b3k, scl_b3k, lm, dl, cc, rv,
                 cov=cov3k, plotlike=True, plotcls=True, 
                 verbose=True, sample_variance=True, mylabel='End-to-End 3K')
###########################################################
title(r'$\ell_m = {} \;-\; \Delta_\ell = {} \;-\; Cov_c = {}$'.format(lm, dl, cc))

subplot(1,2,1)
ylim(5e-4, 0.2)

In [ ]:
signoise = 75.
alllmin = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
alldelta_ell = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
allcovcut = [0.1, 0.15, 0.2, 0.25, 0.3]
# alllmin = [10, 15, 20, 25, 30]
# alldelta_ell = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
# allcovcut = [0.1, 0.15, 0.2, 0.25, 0.3]

# #### Simus sans corrélation spatiale
# dirsims = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/FastSimsNoiseLyon/FastSimNoSpaceCorr/'
# prefix = 'MC_MCFastNoise_n_1000_sig_75.0'
# tit='No Space Correlation'

# #### Simus avec alpha = 1
# dirsims = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/FastSimsNoiseLyon/FastSimAlpha_1/'
# prefix = 'MC_MCFastNoise_n_1000_sig_75.0'
# tit='alpha=1'

#### Simus avec Alpha = 4.5
dirsims = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/FastSimsNoiseLyon/FastSimAlpha_4.5/'
prefix = 'MC_MCFastNoise_n_1000_sig_75.0'
tit='alpha=4.5'

#### Choose which simulation set
nside = 256
lmax = 2 * nside - 1
rv = np.linspace(0,1.,1000)

allrlim = np.zeros((len(alllmin), len(allcovcut), len(alldelta_ell)))


ratio=[]
lm = []
cc = []
dl = []
lv = []

for ilm in range(len(alllmin)):
    lmin = alllmin[ilm]
    print('ell_min = {}'.format(lmin))
    figure()
    ipl=0
    for icc in range(len(allcovcut)):
        for idl in range(len(alldelta_ell)):
            #### Fast Simu
            leff,mcl,scl = read_one_lyon(dirsims, prefix, 
                        alllmin[ilm], alldelta_ell[idl], allcovcut[icc], name='_Flat')
            if leff is -1:
                allrlim[ilm, icc, idl] = np.nan
            else:
                like, cumint, allrlim[ilm, icc, idl] = explore_like(leff, mcl, scl, alllmin[ilm], alldelta_ell[idl], 
                                                                    allcovcut[icc], 
                                                                    rv, cov=cov, sample_variance=True)            
    
      
 

In [ ]:
rc('figure', figsize=(15, 10))
rc('font', size=12)

for ilm in range(len(alllmin)):
    subplot(4,3,ilm+1)
    myrlim = allrlim[ilm,:,:]
    #imshow(myrlim, origin='lower',vmin=np.min(myrlim),vmax=np.max(myrlim))
    imshow(myrlim, origin='lower',vmin=0.0,vmax=0.04, aspect='auto')
    #contourf(myrlim, levels=linspace(0.008, 0.02, 10))
    mini = np.argwhere(myrlim == np.nanmin(myrlim))
    plot(mini[0][1], mini[0][0], 'rx' ,ms=10, markeredgewidth=4,
         label ='$\sigma(r)={0:6.4f}$'.format(np.nanmin(myrlim)))
    plt.xticks(range(len(alldelta_ell)), alldelta_ell)
    plt.yticks(range(len(allcovcut)), allcovcut)
    xlabel('$\Delta\\ell$')
    ylabel('Cov. Threshold')
    title('$\\ell_m$={} - '.format(alllmin[ilm])+tit)
    legend(loc='upper left')
    colorbar()

tight_layout()


In [ ]:
rc('figure', figsize=(15, 8))
rc('font', size=10)

for ilm in range(len(alllmin)):
    subplot(3,4,ilm+1)
    myrlim = allrlim[ilm,:,:]
    imshow(myrlim, origin='lower',vmin=0.008,vmax=0.02, interpolation='lanczos', aspect='auto')
    mini = np.argwhere(myrlim == np.nanmin(myrlim))
    plot(mini[0][1], mini[0][0], 'rx' ,ms=10, markeredgewidth=4,
         label ='$\sigma(r)={0:6.4f}$'.format(np.nanmin(myrlim)))
    plt.xticks(range(len(alldelta_ell)), alldelta_ell)
    plt.yticks(range(len(allcovcut)), allcovcut)
    xlabel('$\Delta\\ell$')
    ylabel('Cov. Threshold')
    title('$\\ell_m$={} - '.format(alllmin[ilm])+tit)
    legend(loc='upper left')
    colorbar()

tight_layout()


### Now we can prepare plots for the papers


In [ ]:
    
lm = 40
dl = 30
cc = 0.1

#### Simus avec Alpha = 4.5
dirsims = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/FastSimsNoiseLyon/FastSimAlpha_4.5/'
prefix = 'MC_MCFastNoise_n_1000_sig_75.0'


leff,mcl,scl = read_sims_lyon(dirsims, prefix, 
                                lm, dl, cc, type='flat')
like, cumint, rlim68, rlim95= explore_like(leff, scl*0, scl, lm, dl, cc, rv,
                                 cov=cov, plotlike=True, plotcls=True, 
                                 verbose=True, sample_variance=True, mylabel=all_titles[i], otherp=0.95)

In [ ]:
def plot_errors_lines(leff, err, dl, color='r', label=''):
    for i in range(len(leff)):
        if i==0:
            plot([leff[i]-dl/2, leff[i]+dl/2], [err[i,s], err[i,s]],color, label=label)
        else:
            plot([leff[i]-dl/2, leff[i]+dl/2], [err[i,s], err[i,s]],color)
        if i < (len(leff)-1):
            plot([leff[i]+dl/2,leff[i]+dl/2], [err[i,s], err[i+1,s]], color)
    

lll = np.arange(512)
cl0 = qc.get_Dl_fromlib(lll, 0, lib=camblib, unlensed=False)[0]   
cl0_01 = qc.get_Dl_fromlib(lll, 0.01, lib=camblib, unlensed=False)[0]   
cl0_06 = qc.get_Dl_fromlib(lll, 0.06, lib=camblib, unlensed=False)[0]   
    
rc('figure', figsize=(15, 8))
rc('font', size=16)
clname = ['TT', 'EE', 'BB', 'TE']

for s in range(4):
    subplot(2,2,s+1)
    plot(lll, np.abs(cl0[:,s]),label='$\Lambda CDM$ r=0')
    if s==2:
        plot(lll, np.abs(cl0_01[:,s]),label='$\Lambda CDM$ r=0.01')
        plot(lll, np.abs(cl0_06[:,s]),label='$\Lambda CDM$ r=0.06')
    plot_errors_lines(leff, scl, dl, 'r', label='QUBIC Noise')
    xlim(0,512)
    ylim(np.min(scl[:,s])/5,np.max([np.max(inputcl[:512,s]*1.5),1.2*np.max(scl[:,s])]))
    yscale('log')
    xlabel('$\\ell$')
    ylabel('$D_\\ell$({})'.format(clname[s]))
    legend(loc='lower right')
    title('QUBIC {} - {} years'.format(d['config'], d['effective_duration']))
tight_layout()

In [ ]:
rc('figure', figsize=(15, 8))
rc('font', size=15)
subplot(1,2,1)
s=2
plot(lll, np.abs(cl0[:,s]),label='r=0')
if s==2:
    plot(lll, np.abs(cl0_01[:,s]),label='r=0.01')
    plot(lll, np.abs(cl0_06[:,s]),label='r=0.06')
plot_errors_lines(leff, scl, dl, 'r', label='QUBIC {} - {} years'.format(d['config'], d['effective_duration']))
title(r'$\ell_m = {} \;-\; \Delta_\ell = {} \;-\; Cov_c = {}$'.format(lm, dl, cc))
xlim(0,512)
ylim(np.min(scl[:,s])/5,np.max([np.max(inputcl[:512,s]*1.5),1.2*np.max(scl[:,s])]))
yscale('log')
xlabel('$\\ell$', fontsize=17)
ylabel('$D_\\ell$({})'.format(clname[s]), fontsize=17)
legend(loc='lower right')

subplot(1,2,2)
plot(rv,like, label='QUBIC {} - {} years'.format(d['config'], d['effective_duration']))
plot([rlim68, rlim68],[0,1.1],'k--', label='r < {0:5.3f} at 68% C.L.'.format(rlim68))
plot([rlim95, rlim95],[0,1.1],'k:', label='r < {0:5.3f} at 95% C.L.'.format(rlim95))
annotate("BKP-2018\n 95% C.L.", xy=(0.06, 0.01), xytext=(0.06, 0.2),
            arrowprops=dict(width=10, headwidth=20), ha='center')
title(r'$\ell_m = {} \;-\; \Delta_\ell = {} \;-\; Cov_c = {}$'.format(lm, dl, cc))
legend()
xlabel('r')
ylabel('Posterior')
xlim(0,0.07)
ylim(0,1.1)